In [1]:
# Importem les llibreries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import xarray as xr # per guardar les dades 3D
from collections import Counter

In [2]:
# Carreguem les dades
data_df = pd.read_csv('results.csv')

# Emplenem els espais en blanc amb 0
data_df = data_df.fillna(0.)

In [62]:
# Read the number of games played per each player
dataarray = xr.open_dataset('stats.nc', engine='scipy')
played_games = dataarray['GamesPlayed'].isel(matchday=-1)

In [3]:
# Obtenim una llista amb tots els noms dels participants
players_names = np.unique(data_df[['Jugador 1', 'Jugador 2', 'Jugador 3', 'Jugador 4']].values.flatten())

# Llista de dies jugats
matchdays = pd.unique(data_df['D'])

In [72]:
39 / played_games.sel(player='Pau').values

1.0

In [91]:
# DataFrame with the number of team occurrences
mates_df = pd.DataFrame(columns = players_names) # number of games played with each mate
matesplayed_df = pd.DataFrame(columns = players_names) # number of games played with each mate divided by number of games playe by player

for player in players_names: # for eack player
    # Initialize a dictionary where the number of team-ups will be stored
    mate_count = {}
    mateplayed_count = {} # number of games played with mate divided by total games played
    for mate in players_names:
        mate_count[mate] = 0 # set counter to 0
        mateplayed_count[mate] = 0

    # Get the list of all teammates
    lineups = [['Jugador 1', 'Jugador 2'], ['Jugador 2', 'Jugador 1'], ['Jugador 3', 'Jugador 4'], ['Jugador 4', 'Jugador 3']]
    for lineup in lineups:
        teammate_list = matchday_df[lineup[1]][matchday_df[lineup[0]] == player]
        teammate_list_counts = teammate_list.value_counts() # count how many team occurences
        for mate in teammate_list_counts.keys():
            mate_count[mate] = mate_count.get(mate, 0) + teammate_list_counts.loc[mate] # store the teammate join number

    for mate in mate_count.keys():   
        if played_games.sel(player = player).values == 0:
            continue
        else:
            mateplayed_count[mate] = mate_count.get(mate, 0) / played_games.sel(player = player).values

    mates_df = pd.concat([mates_df, pd.DataFrame([mate_count])])
    matesplayed_df = pd.concat([matesplayed_df, pd.DataFrame([mateplayed_count])])#.reset_index(drop=True)

# transpose to match dimesnions with xarray dimensions (x: player, y: teammate)
matesplayed_df = matesplayed_df.transpose()

# set index to teammate name
mates_df = mates_df.set_index(players_names)


C:\Users\Admin\AppData\Local\Temp\ipykernel_19576\1790517054.py:28: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  matesplayed_df = pd.concat([matesplayed_df, pd.DataFrame([mateplayed_count])])#.reset_index(drop=True)


In [90]:
# Creem una DataArray de xarray. Hi especifiquem els noms de cada dimensió
mates_da = xr.DataArray(mates_df.values, dims = ('teammate', 'player'),
                                      coords = {'teammate': mates_df.index, 'player': mates_df.columns})
matesplayed_da = xr.DataArray(matesplayed_df.values, dims = ('teammate', 'player'),
                                      coords = {'teammate': mates_df.index, 'player': mates_df.columns})

# Combinem tots els DataArrays a un únic Dataset de xarray (cal que les coords siguin les mateixes per a tots)
dataset = xr.Dataset({"Teammates": mates_da,
                      "TeammatesPlayed":matesplayed_da})

# TODO: el procés de crear el DataArray a partir del DataFrame es pot automatitzar amb una funció que faci un concat al dataframe. 

# dataset['goals'] = goals_da # si volem afegir un nou element
dataset

<xarray.Dataset> Size: 4kB
Dimensions:          (teammate: 15, player: 15)
Coordinates:
  * teammate         (teammate) object 120B 'Alejandro' 'Antía' ... 'Víctor'
  * player           (player) object 120B 'Alejandro' 'Antía' ... 'Víctor'
Data variables:
    Teammates        (teammate, player) object 2kB 0 0 0 0 0 0 0 ... 1 0 0 0 0 0
    TeammatesPlayed  (teammate, player) float64 2kB 0.0 0.0 0.0 ... 0.0 0.0 0.0